In [8]:
import sys
import os
import dolfin
sys.path.append(os.getcwd()[:-6])

In [3]:
from src.model import PerfusionGasExchangeModel
from src.params import params

In [4]:
folder = "test-folder-prescribed-u"
path = os.path.join("../raw-data", folder)

In [10]:
model = PerfusionGasExchangeModel(folder_path=path, params=params)
model.generate_slab_mesh(
    dims=(200, 6, 200), elems=(40, 3, 30), save=True, periodic=True
)
model.sim_p(save=True)
model.sim_bst(final_time=1, num_steps=5, save=True)

Solving linear variational problem.
No Jacobian form specified for nonlinear variational problem.
Differentiating residual form F to obtain Jacobian J = F'.


In [9]:
model.boundaries
boundaries = dolfin.File(model.folder_path+'/bnd/bnd.pvd')
boundaries << model.boundaries